<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# AI-First Finance

**Multi Layer Perceptron and Deep Neural Networks**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')

## Universal Approximation

### The Data

In [ ]:
def normalize(x):
    return (x - x.mean()) / x.std()
def unit(x):
    return (x - x.min()) / (x.max() - x.min())

In [ ]:
x = np.linspace(0, 5 * np.pi, 100)
y = np.cos(x)

In [ ]:
x = normalize(x)
y = normalize(y)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro');

### With Scikit-Learn

In [ ]:
from sklearn.neural_network import MLPRegressor, MLPClassifier

In [ ]:
model = MLPRegressor(hidden_layer_sizes=1 * [1024,],
                     activation='relu', solver='adam',
                     learning_rate_init=0.001, nesterovs_momentum=False,
                     shuffle=False, max_iter=10000,
                     validation_fraction=0.1)

In [ ]:
%time model.fit(x.reshape(-1, 1), y)

In [ ]:
pred = model.predict(x.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro', label='original data')
plt.plot(x, pred, label='prediction')
plt.legend();

### With Keras

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(4 * 24, input_dim=1, activation='relu'))
model.add(Dense(4 * 24, activation='relu'))

In [ ]:
model.add(Dense(1, activation='linear'))

Original paper on the Adam optimizer: https://arxiv.org/pdf/1412.6980v8.pdf.

In [ ]:
adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9,
                                    beta_2=0.999, amsgrad=False)

In [ ]:
model.compile(loss='mse', optimizer=adam, metrics=['mse', 'accuracy'])

In [ ]:
model.summary()

In [ ]:
%time model.fit(x, y, epochs=2000, verbose=False)

In [ ]:
scores = model.evaluate(x, y)

In [ ]:
print('mse: %.5f' % (scores[1]))

In [ ]:
pred = model(tf.convert_to_tensor(x, dtype=tf.float32), training=False).numpy()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro', label='original data')
plt.plot(x, pred, label='prediction')
plt.legend();

## Estimation

### Features & Labels

In [ ]:
features = 5
samples = 5

In [ ]:
np.random.seed(10)
l0 = np.random.random((samples, features))
l0[:5]  # input layer (features)

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.random(samples) # labels
y

### OLS Regression

In [ ]:
reg = np.linalg.lstsq(l0, y, rcond=-1)[0]

In [ ]:
reg

In [ ]:
np.allclose(np.dot(l0, reg), y)

In [ ]:
res = pd.DataFrame({'y': y, 'pred': np.dot(l0, reg)},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

### Scikit-Learn

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(32),
                     max_iter=10000,
                     learning_rate_init=0.01)

In [ ]:
model.fit(l0, y)

In [ ]:
mean_squared_error(model.predict(l0), y)

In [ ]:
res = pd.DataFrame({'y': y, 'pred': model.predict(l0)},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

### Keras

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=l0.shape[1], activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [ ]:
%time model.fit(l0, y, epochs=1000, verbose=False)

In [ ]:
mean_squared_error(model.predict(l0), y)

In [ ]:
res = pd.DataFrame({'y': y, 'pred': model.predict(l0).flatten()},
                   index=range(len(y)))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

## Classification

### Features & Labels

In [ ]:
features = 5
samples = 10

In [ ]:
np.random.seed(3)
l0 = np.random.randint(0, 2, (samples, features))
l0  # input layer (features)

In [ ]:
np.linalg.matrix_rank(l0)

In [ ]:
y = np.random.randint(0, 2, (samples))   # labels
y

### OLS Regression

In [ ]:
reg = np.linalg.lstsq(l0, y, rcond=-1)[0]

In [ ]:
reg

In [ ]:
np.dot(l0, reg).round() == y

### Scikit-Learn

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(16),
                     learning_rate_init=0.1)

In [ ]:
model.fit(l0, y)

In [ ]:
model.predict(l0)

In [ ]:
model.predict(l0).round() == y

### Keras

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=l0.shape[1], activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [ ]:
%time model.fit(l0, y, epochs=1000, verbose=False)

In [ ]:
model.predict(l0).flatten().round() == y

## Vectorized Backtesting &mdash; Scikit-Learn

### The Data

In [ ]:
url = 'http://hilpisch.com/tr_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
lags = 10
sym = 'GLD'
data = pd.DataFrame(raw[sym])
data['r'] = np.log(data / data.shift(1))
data['d'] = np.where(data['r'] > 0, 1, 0)

In [ ]:
cols = []
for lag in range(1, lags+1):
    col = f'lag_{lag}'
    data[col] = data['d'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)

In [ ]:
train = data.iloc[-1500:-500]

In [ ]:
test = data.iloc[-500:].copy()

### Fitting

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model = MLPClassifier(hidden_layer_sizes=3 * [128,],
                      max_iter=5000, random_state=1)

In [ ]:
l0 = normalize(train[cols])
y = train['d']

In [ ]:
%time model.fit(l0, y)

In [ ]:
accuracy_score(model.predict(l0), train['d'])

### Backtesting

In [ ]:
l0_ = normalize(test[cols])
y_ = test['d']

In [ ]:
test['pred'] = model.predict(l0_).round()

In [ ]:
test['p'] = np.where(test['pred'] == 1, 1, -1)

In [ ]:
test['p'].value_counts()

In [ ]:
sum(test['p'].diff() != 0)

In [ ]:
test['s'] = test['p'] * test['r']

In [ ]:
test[['r', 's']].sum().apply(np.exp)

In [ ]:
test[['r', 's']].cumsum().apply(np.exp).plot(figsize=(10, 6));

## Vectorized Backtesting &mdash; Keras

### The Data

In [ ]:
test = data.iloc[-500:].copy()

### Fitting

In [ ]:
model = Sequential()
model.add(Dense(256, input_dim=l0.shape[1], activation='sigmoid'))
# model.add(Dense(256, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'acc'])

In [ ]:
# model.fit?

In [ ]:
%%time
history = model.fit(l0, y, epochs=1000, verbose=False,
               validation_split=0.25, shuffle=False)

In [ ]:
accuracy_score(model.predict(l0).round(), train['d'])

In [ ]:
res = pd.DataFrame(history.history)

In [ ]:
res.tail(3)

In [ ]:
ax = res.iloc[2:].plot(figsize=(10, 6), style=['--', '--', '-', '-'])
x = np.arange(len(res));
reg = np.polyfit(x, res['val_acc'], deg=2)
plt.plot(x, np.polyval(reg, x), 'r');

### Backtesting

In [ ]:
l0_ = normalize(test[cols])
y_ = test['d']

In [ ]:
test['pred'] = model.predict(l0_).round()

In [ ]:
sum(test['pred'] == y_) / len(y_)

In [ ]:
test['p'] = np.where(test['pred'] == 1, 1, -1)

In [ ]:
test['p'].value_counts()

In [ ]:
sum(test['p'].diff() != 0)

In [ ]:
test['s'] = test['p'] * test['r']

In [ ]:
test[['r', 's']].sum().apply(np.exp)

In [ ]:
test[['r', 's']].cumsum().apply(np.exp).plot(figsize=(10, 6));

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">